In [1]:
import numpy as np
import copy  as cpy
import queue
test=0
test_100=0
test_150=0
test_200=0
new_inst_tst=1
best_improvement=1
first_improvement=0
feasable_zone=1
infeasable_zone=0


In [2]:
def read_txt2(fileLocation):
    with open(fileLocation,'r') as file:
        dimSubset,dimGroundSet,maxBudget = [int(i) for i in file.readline().split()] 
        # weights
        subsets_weights=np.array( [int(i) for i in file.readline().split()])
        #print(dimSubset,dimGroundSet,maxBudget)
        # profits
        ground_set_profits=np.array( [int(i) for i in file.readline().split()])
        objPerSubset={}
        for sub in range(dimSubset):
            objPerSubset[sub]=[]
        relation_matrix=np.array( [int(i) for i in file.readline().split()])
        for sub in range(dimSubset):
            for obj in range(sub*dimGroundSet,(sub+1)*dimGroundSet):
                if relation_matrix[obj]==1:
                    objPerSubset[sub].append(obj-dimGroundSet*sub)
        return objPerSubset,subsets_weights,ground_set_profits,maxBudget

#(objPerSubset,subsets_weights,ground_set_profits,maxBudget)=read_txt2("./instances2/bmcp_1100_1000_0.3_3000.txt")
#print(objPerSubset)

In [3]:
# calculate the score of a subset
def calcSubsetScore(subsetID,subsetWeight,subsetObjs,subsetPerObj,objScores):
    # Score= Sum of Value of Objects ( not yet added) /weight of the subset
    score=0
    for obj in subsetObjs: 
        if len(subsetPerObj[obj])==0:
            score+=objScores[obj]
    return score/subsetWeight[subsetID]


In [4]:
# until we have budget ( remaining_budget> max(not selected subsets)) 
#  Find the best subset 
#  Update the score the solution and budget
def greedy(subsets,subsetWeights,objScores, budget):
    # generate the list of all the subsets
    subset_ids=list(range(len(subsets)))
    #print(type(subset_ids[0]))
    # last element == the most expensive 
    subset_ids.sort(key=lambda x:subsetWeights[x])
    
    #print("ordered for wrights")
    #for sub in subset_ids:
    #    print("id",sub,"w",subsetWeights[sub])
    
    # init the map of sunsets per objects
    subsetPerObj={}
    remaining_budget=budget
    for obj in range(len(objScores)):
        subsetPerObj[obj]=[]
    # init the score
    score=0
    #init the solution( the list of selected subs)
    solution=np.array([])
    # helper vars
    (best_score,best_sub,curr_score)=(-1,0,0)
    while 1:
        best_score,curr_score=0,0
        for sub in subset_ids:
            curr_score=calcSubsetScore(sub,subsetWeights,subsets[sub],subsetPerObj,objScores)
            # if the score is better
            if curr_score>best_score:
                # update the score
                best_score=curr_score
                # save the id
                best_sub=sub
        if best_score==0:
            #print("No need to update")
            break
        #print("Adding ",best_sub)
        # update the score and the subs per book
        for obj in subsets[best_sub]:
            if len(subsetPerObj[obj])==0:
                score+=objScores[obj]
            subsetPerObj[obj].append(best_sub)
        #insert the element in the solution
        solution=np.append(solution,best_sub)
        # remove the element
        #print("removing",best_sub)
        subset_ids.remove(best_sub)
        #print("Remaining subs" ,subset_ids)
        # update the budg
        budget-=subsetWeights[best_sub]
        # stop if we've ended the subsets or the budget
        if len(subset_ids)==0:
            #print("Ended")
            break
        if budget<subsetWeights[subset_ids[len(subset_ids)-1]]:
            #print(" Remainig budget", budget," with cap", subsetWeights[subset_ids[len(subset_ids)-1]])
            break
            
    return (score,subsetPerObj,solution.astype(int))
    

In [5]:
if new_inst_tst:
    (objPerSubset,subsets_weights,ground_set_profits,maxBudget)=read_txt2("./instances2/bmcp_1100_1000_0.3_3000.txt")
    (score,subsetPerObj,solution)=greedy(objPerSubset,subsets_weights,ground_set_profits,maxBudget)
    print(" The greedy score", score)
    tot_weig=sum(subsets_weights[s] for s in solution)
    if tot_weig<=maxBudget:
        print("Feasable solution")
    else:
        print("Infeasable")
    print(tot_weig,maxBudget,sum(subsets_weights))
    print(solution)
    print("Checking validity")
    k=0
    for to_check in solution:
        for obj in objPerSubset[to_check]:
            if not to_check in subsetPerObj[obj]:
                k=1
    if k:
        print("Invalid")
    else:
        print("valid")

 The greedy score 136702
Feasable solution
2841 3000 246511
[1054  164  630  508  773 1031  761 1016   14  309 1060 1078  915   67
  600  746  170]
Checking validity
valid


In [27]:
def localSearch(score,solution,subsetPerObj,objPerSubset,subsetWeights,objScores,budget,zone,tabu_list):
    restorer_budget=sum(subsetWeights[sub] for sub in solution)
    to_insert_cost,to_rmv_cost,new_score=0,0,0
    to_rmv_id,to_insert_id=0,0
    to_return_cost,to_return_score,swapped=0,-1,(0,0)
    removed,inserted_cost=0,0
    found=0
    # for each subset not in list 
    for sub in range(len(subsetWeights)):
        if not sub in solution:
            
            to_insert_id,to_insert_cost=sub,subsetWeights[sub]
            # try to swap it with another subset
            for k in range(len(solution)):
                if not solution[k] in list(tabu_list.queue):
                    to_rmv_id,to_rmv_cost=solution[k],subsetWeights[solution[k]]
                    current_budget=restorer_budget
                    # try to swap it calculating the new score
                    #print("Actual cost", new_cost, "To remove", to_rmv_cost,"To add",to_insert_cost)
                    old_budget=current_budget
                    current_budget=current_budget-to_rmv_cost+to_insert_cost
                    if current_budget==0:
                        print("Old budget",current_budget, "Removed",to_rmv_cost,"Inserted",to_insert_cost)
                    if zone==infeasable_zone or (zone==feasable_zone and budget>=new_cost):
                        # always to init with the score
                        new_score=score
                        # for updating the score we need :
                        # 1-remove all the obj from the leaving subset
                        # I won't remove it but only check if the removed sub was the only one with the obj
                        # and in case update the score
                        for obj in objPerSubset[to_rmv_id]:
                            # in case it was the only subset with the object
                            if len(subsetPerObj[obj])==1:
                                new_score-=objScores[obj]
                        # 2- To add the score of the new objects added from the inserted subs
                        for obj in objPerSubset[to_insert_id]:
                            # in case the object wasn't in list  then update the score
                            if len(subsetPerObj[obj])==0:
                                new_score+=objScores[obj]
                        # sub the penality factor
                        new_score-=int(budget/current_budget)*max(0,current_budget-budget)
                        if new_score>to_return_score:
                            # REMEMBER : SWAPPED[0] IS THE IDX IN SOLUTION ARRAY, NOT THE ID
                            swapped=(k,sub)
                            to_return_score=new_score
                            found=1
                            to_return_cost=current_budget
    if found==0:
        print("ENDED")
    #print(" old cost ", current_budget, "new ",to_return_cost," budget ",budget)
    # now make the actual swap
    to_rmv_id=solution[swapped[0]]
    to_insert_id=swapped[1]
    #print("removing", to_rmv_id,"Inserting",to_insert_id, " new cost",to_return_cost," removed",to_rmv_cost, "inserted",to_insert_cost)
    # 1-remove all the obj from the leaving subset 
    for obj in objPerSubset[to_rmv_id]:
        subsetPerObj[obj].remove(to_rmv_id)
    # 2- Add the new objects
    for obj in objPerSubset[to_insert_id]:
        subsetPerObj[obj].append(to_insert_id)
    solution[swapped[0]]=to_insert_id
    # update the tabu list 
    if tabu_list.full():
        tabu_list.get()
        tabu_list.put(to_rmv_id)
    else:
        tabu_list.put(to_rmv_id)
    # and finally end
    return int(to_return_score),subsetPerObj,solution,to_return_cost,found
                        

In [32]:
# Problem data : objPerSubset,subsetWeights,objScores,budget
# Problem solution: score,subsetPerObj,solution
def TabuSearch(objPerSubset,subsetWeights,objScores,budget,score,subsetPerObj,solution,Nt,Ni,tabu_list_size):
    iters_without_updating=0
    iters_infeasable=0
    # current solution values used during tabu search
    current_budget=sum(subsetWeights[sub] for sub in solution)
    curr_score=score
    curr_solution=solution.copy()
    curr_subsetPerObj=cpy.deepcopy(subsetPerObj)
    # Best values found in the search
    best_budget=current_budget
    best_score=score
    best_solution=curr_solution.copy()
    best_subsetPerObj=cpy.deepcopy(curr_subsetPerObj)
    tabu_list=queue.Queue(tabu_list_size)
    
    #new_score,new_subset_per_obj,new_solution,new_budget=0,{},[],0
    while iters_without_updating<Nt:

        curr_score,curr_subsetPerObj,curr_solution,current_budget,found=localSearch(curr_score,
                                                                                    curr_solution,curr_subsetPerObj,
                                                                                    objPerSubset,subsetWeights,objScores,
                                                                                    budget,infeasable_zone,tabu_list)
        if not found:
            break
        # in case it is infeasable update the budget
        if current_budget> budget:
            iters_infeasable+=1
            #print("Infeasable , budget",budget," weig",current_budget," score",best_score)
        else:
            #print("Feasable , budget",budget," weig",current_budget," score",best_score)
            iters_infeasable=0 
            # in case the solution is feasable and better then update
            if curr_score> best_score:
                iters_without_upating=0
                best_budget=current_budget
                best_score=curr_score
                best_solution=curr_solution.copy()
                best_subsetPerObj=cpy.deepcopy(curr_subsetPerObj)
            else:
                iters_without_updating+=1
    print(iters_without_updating)
    return (best_score,best_subsetPerObj,best_solution)

In [33]:
if new_inst_tst:
    (objPerSubset,subsetWeights,objScores,budget)=read_txt2( "./instances2/bmcp_1100_1000_0.3_3000.txt")
    (greedy_score,greedy_subsetPerObj,greedy_solution)=greedy(objPerSubset,subsetWeights,objScores,budget)
    print(" The greedy score", greedy_score)
    tot_weig=sum(subsetWeights[s] for s in greedy_solution)
    if tot_weig<=budget:
        print("Feasable solution")
    else:
        print("Infeasable")
    (tabu_score,tabu_subsetPerObj,tabu_solution)=TabuSearch(objPerSubset,subsetWeights,objScores,budget,greedy_score,greedy_subsetPerObj,greedy_solution,100,50,20)
    print("Tabu score",tabu_score)
    tot_weig=sum(subsetWeights[s] for s in tabu_solution)
    if tot_weig<=budget:
        print("Feasable solution")
    else:
        print("Infeasable")
    print("weig",tot_weig)
    k=0
    for to_check in tabu_solution:
        for obj in objPerSubset[to_check]:
            if not to_check in tabu_subsetPerObj[obj]:
                k=1
    if k:
        print("Invalid")
    else:
        print("valid")
    

 The greedy score 136702
Feasable solution
ENDED
0
Tabu score 138417
Feasable solution
weig 2925
valid


In [34]:
if new_inst_tst:
    (objPerSubset,subsetWeights,objScores,budget)=read_txt2( "./instances2/bmcp_1100_1000_0.5_2000.txt")
    (greedy_score,greedy_subsetPerObj,greedy_solution)=greedy(objPerSubset,subsetWeights,objScores,budget)
    print(" The greedy score", greedy_score)
    tot_weig=sum(subsetWeights[s] for s in greedy_solution)
    if tot_weig<=budget:
        print("Feasable solution")
    else:
        print("Infeasable")
    (tabu_score,tabu_subsetPerObj,tabu_solution)=TabuSearch(objPerSubset,subsetWeights,objScores,budget,greedy_score,greedy_subsetPerObj,greedy_solution,500,50,20)
    print("Tabu score",tabu_score)
    tot_weig=sum(subsetWeights[s] for s in tabu_solution)
    if tot_weig<=budget:
        print("Feasable solution")
    else:
        print("Infeasable")
    print("weig",tot_weig)
    k=0
    for to_check in tabu_solution:
        for obj in objPerSubset[to_check]:
            if not to_check in tabu_subsetPerObj[obj]:
                k=1
    if k:
        print("Invalid")
    else:
        print("valid")

    

 The greedy score 120386
Feasable solution
ENDED
4
Tabu score 122190
Feasable solution
weig 1895
valid
